In [1]:
import pandas as pd

In [2]:
file = 'Bank.csv'
bank = pd.read_csv(file, delimiter=';', decimal = '.')

for cols in bank.columns:
    print(cols, ': ', bank[cols].dtype)

('age', ': ', dtype('float64'))
('job', ': ', dtype('O'))
('marital', ': ', dtype('O'))
('education', ': ', dtype('O'))
('default', ': ', dtype('O'))
('housing', ': ', dtype('O'))
('loan', ': ', dtype('O'))
('contact', ': ', dtype('O'))
('month', ': ', dtype('O'))
('day_of_week', ': ', dtype('O'))
('duration', ': ', dtype('float64'))
('campaign', ': ', dtype('float64'))
('pdays', ': ', dtype('O'))
('previous', ': ', dtype('O'))
('poutcome', ': ', dtype('O'))
('emp.var.rate', ': ', dtype('O'))
('cons.price.idx', ': ', dtype('float64'))
('cons.conf.idx', ': ', dtype('float64'))
('euribor3m', ': ', dtype('O'))
('nr.employed', ': ', dtype('O'))
('y', ': ', dtype('O'))


In [3]:
#Fixing the inputting error in obs 650
length = len(bank.columns)
mask_div = bank['marital'] == 'divorceded”;”basic.9y'

for col in reversed(range(3,length)):
    bank.loc[mask_div, bank.columns[col]] = bank.loc[mask_div, bank.columns[col-1]]

bank.loc[mask_div, 'marital'] = 'divorced'
bank.loc[mask_div, 'education'] = 'basic.9y'

In [4]:
#Fixing all errors in obs 574
mask_yess = bank['default'] =='yess”;”yes'
for col in reversed(range(7,length)):
    bank.loc[mask_yess, bank.columns[col]] = bank.loc[mask_yess, bank.columns[col-2]]
    
bank.loc[mask_yess,'education'] = 'basic.6y'
bank.loc[mask_yess,'default'] = 'no'
bank.loc[mask_yess,'housing'] = 'yes'
bank.loc[mask_yess,'loan']  ='yes'

In [5]:
#fixing the error in obs 569
bank['default'].value_counts()
mask_def = bank['default'] == 'noo”;”yes'
bank.loc[mask_def]
for col in reversed(range(5,length)):
    bank.loc[mask_def, bank.columns[col]] =  bank.loc[mask_def, bank.columns[col-1]]
bank.loc[mask_def, 'default'] = 'no'
bank.loc[mask_def, 'housing'] = 'yes'

In [6]:
#fixing all errors and replacing 'duration' for obs 678
bank[bank.columns[9]].value_counts()
mask_day = bank['day_of_week'] == 'thu”;-;1;999;0;nonexistent"'
for col in reversed(range(15,length)):
    bank.loc[mask_day, bank.columns[col]] = bank.loc[mask_day, bank.columns[col-5]]
bank.loc[mask_day, 'day_of_week'] = 'thu'
bank.loc[mask_day, 'duration'] = 0.0 # the original input was '-' which has been chosen to be interpreted as 0
bank.loc[mask_day, 'campaign'] = 1.0
bank.loc[mask_day, 'pdays'] = 999.0
bank.loc[mask_day, 'previous'] = 0.0
bank.loc[mask_day, 'poutcome'] = 'nonexistent'

In [7]:
#ensuring all numeric values are numeric
bank['age'] = pd.to_numeric(bank['age'])
bank['duration'] = pd.to_numeric(bank['duration'])
bank['campaign'] = pd.to_numeric(bank['campaign'])
bank['pdays'] = pd.to_numeric(bank['pdays'])
bank['previous'] = pd.to_numeric(bank['previous'])
bank['emp.var.rate'] = pd.to_numeric(bank['emp.var.rate'])
bank['cons.price.idx'] = pd.to_numeric(bank['cons.price.idx'])
bank['cons.conf.idx'] = pd.to_numeric(bank['cons.conf.idx'])
bank['euribor3m'] = pd.to_numeric(bank['euribor3m'])
bank['nr.employed'] = pd.to_numeric(bank['nr.employed'])

#ensuring data consistency
for cols in bank.columns:
    print(cols,": ", bank[cols].dtype)

('age', ': ', dtype('float64'))
('job', ': ', dtype('O'))
('marital', ': ', dtype('O'))
('education', ': ', dtype('O'))
('default', ': ', dtype('O'))
('housing', ': ', dtype('O'))
('loan', ': ', dtype('O'))
('contact', ': ', dtype('O'))
('month', ': ', dtype('O'))
('day_of_week', ': ', dtype('O'))
('duration', ': ', dtype('float64'))
('campaign', ': ', dtype('float64'))
('pdays', ': ', dtype('float64'))
('previous', ': ', dtype('float64'))
('poutcome', ': ', dtype('O'))
('emp.var.rate', ': ', dtype('float64'))
('cons.price.idx', ': ', dtype('float64'))
('cons.conf.idx', ': ', dtype('float64'))
('euribor3m', ': ', dtype('float64'))
('nr.employed', ': ', dtype('float64'))
('y', ': ', dtype('O'))


In [8]:
#removing any whitespaces and transforming to lowercase
for col in bank.columns:
    if bank[col].dtype == object:
        for obs in bank[col]:
            obs = obs.lower().strip()

In [9]:
#handling missing values
print(bank.isnull().any())

for cols in bank.columns:
    mask_null = bank[cols].isnull() == True
    if bank[cols].isnull().any() == True:
        bank[cols].fillna(bank[cols].mean(), inplace = True)

age                True
job               False
marital           False
education         False
default           False
housing           False
loan              False
contact           False
month             False
day_of_week       False
duration          False
campaign          False
pdays             False
previous          False
poutcome          False
emp.var.rate      False
cons.price.idx     True
cons.conf.idx     False
euribor3m          True
nr.employed        True
y                 False
dtype: bool


In [10]:
#filtering out impossible age values
bank = bank[bank.age<120]
bank = bank[bank.age>0]

In [11]:
#check that all the categorical features have legitmate entries:
for cols in bank.columns:
    if bank[cols].dtype == object:
        print(cols, bank[cols].unique())

('job', array(['blue-collar', 'services', 'admin.', 'entrepreneur',
       'self-employed', 'technician', 'management', 'student', 'retired',
       'housemaid', 'unemployed', 'unknown'], dtype=object))
('marital', array(['married', 'single', 'divorced', 'unknown'], dtype=object))
('education', array(['basic.9y', 'high.school', 'university.degree',
       'professional.course', 'basic.6y', 'basic.4y', 'unknown',
       'illiterate'], dtype=object))
('default', array(['no', 'unknown', 'yes'], dtype=object))
('housing', array(['yes', 'no', 'unknown'], dtype=object))
('loan', array(['no', 'unknown', 'yes'], dtype=object))
('contact', array(['cellular', 'telephone'], dtype=object))
('month', array(['may', 'jun', 'nov', 'sep', 'jul', 'aug', 'mar', 'oct', 'apr',
       'dec'], dtype=object))
('day_of_week', array(['fri', 'wed', 'mon', 'thu', 'tue'], dtype=object))
('poutcome', array(['nonexistent', 'failure', 'success'], dtype=object))
('y', array(['no', 'yes'], dtype=object))


In [12]:
#check that features 11-14 contain non-negative integers
for i in range(10,14):
    mask_a = bank[bank.columns[i]] < 0
    mask_b = bank[bank.columns[i]] % 1 != 0
    print(bank.loc[mask_a,bank.columns[i]])
    print(bank.loc[mask_b,bank.columns[i]])

Series([], Name: duration, dtype: float64)
Series([], Name: duration, dtype: float64)
Series([], Name: campaign, dtype: float64)
Series([], Name: campaign, dtype: float64)
Series([], Name: pdays, dtype: float64)
Series([], Name: pdays, dtype: float64)
Series([], Name: previous, dtype: float64)
Series([], Name: previous, dtype: float64)


In [13]:
#check the statistics of features 15-19
for i in range(15,20):
    print(bank[bank.columns[i]].describe())

count    4117.000000
mean        0.085111
std         1.563084
min        -3.400000
25%        -1.800000
50%         1.100000
75%         1.400000
max         1.400000
Name: emp.var.rate, dtype: float64
count    4117.000000
mean       93.579620
std         0.579291
min        92.201000
25%        93.075000
50%        93.749000
75%        93.994000
max        94.767000
Name: cons.price.idx, dtype: float64
count    4117.000000
mean      -40.498786
std         4.594323
min       -50.800000
25%       -42.700000
50%       -41.800000
75%       -36.400000
max       -26.900000
Name: cons.conf.idx, dtype: float64
count    4117.000000
mean        3.622016
std         1.733125
min         0.635000
25%         1.334000
50%         4.857000
75%         4.961000
max         5.045000
Name: euribor3m, dtype: float64
count    4117.000000
mean     5166.537812
std        73.623481
min      4963.600000
25%      5099.100000
50%      5191.000000
75%      5228.100000
max      5228.100000
Name: nr.employed, d

In [14]:
#saving the modifications as a new csv file
bank.to_csv("bank2.csv")